In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns

# Load and initial preprocessing
df = pd.read_csv('../data/sample-2/random-simulation.csv')

# Convert timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Extract time features
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['Day'] = df['Timestamp'].dt.day
df['Hour'] = df['Timestamp'].dt.hour
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek
df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)

# Handle missing values if any
df = df.dropna()

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")
print(f"Number of stores: {df['StoreID'].nunique()}")
print(f"Number of customers: {df['CustomerID'].nunique()}")
print(f"Number of products: {df['ProductID'].nunique()}")

Dataset shape: (37934, 19)
Date range: 2022-01-01 09:04:39 to 2022-06-30 20:55:44
Number of stores: 8
Number of customers: 1000
Number of products: 50


## DBSCAN

In [6]:
# Aggregate by Store and Date
daily_store_metrics = df.groupby(['StoreID', df['Timestamp'].dt.date]).agg({
    'TotalPrice': ['sum', 'mean', 'count'],
    'CustomerID': 'nunique',
    'ProductID': 'nunique',
    'Quantity': 'sum',
    'DiscountAmount': 'sum'
}).reset_index()

# Flatten column names
daily_store_metrics.columns = ['StoreID', 'Date', 'total_revenue', 'avg_transaction_value', 
                              'transaction_count', 'unique_customers', 'unique_products', 
                              'total_quantity', 'total_discounts']

# Add derived features
daily_store_metrics['revenue_per_customer'] = daily_store_metrics['total_revenue'] / daily_store_metrics['unique_customers']
daily_store_metrics['items_per_transaction'] = daily_store_metrics['total_quantity'] / daily_store_metrics['transaction_count']

In [28]:
display(daily_store_metrics.head())

,StoreID,Date,total_revenue,avg_transaction_value,transaction_count,unique_customers,unique_products,total_quantity,total_discounts,revenue_per_customer,items_per_transaction
0,1,2022-01-01,6510.12,171.318947,38,12,27,75,150.02,542.510000,1.973684
1,1,2022-01-02,3683.01,175.381429,21,10,17,41,112.70,368.301000,1.952381
2,1,2022-01-03,1943.75,161.979167,12,5,12,23,33.36,388.750000,1.916667
3,1,2022-01-04,6051.44,208.670345,29,11,21,63,7.84,550.130909,2.172414
4,1,2022-01-05,1843.92,204.880000,9,5,7,20,23.53,368.784000,2.222222


In [9]:
import os

os.makedirs('../data/sample-2/processed', exist_ok=True)
daily_store_metrics.to_csv('../data/sample-2/processed/daily_store_metrics.csv', index=False)

## Time Series Clustering Preparation

In [10]:
# Create time series features
df['Date'] = df['Timestamp'].dt.date
df['Hour'] = df['Timestamp'].dt.hour
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek
df['Month'] = df['Timestamp'].dt.month

# Daily time series by store
daily_ts = df.groupby(['StoreID', 'Date']).agg({
    'TotalPrice': 'sum',
    'CustomerID': 'nunique',
    'TransactionID': 'nunique'
}).reset_index()

# Pivot to create time series matrix (rows=stores, columns=dates)
revenue_ts_matrix = daily_ts.pivot(index='StoreID', columns='Date', values='TotalPrice').fillna(0)

# Add seasonal features
hourly_patterns = df.groupby(['StoreID', 'Hour'])['TotalPrice'].sum().unstack(fill_value=0)
weekly_patterns = df.groupby(['StoreID', 'DayOfWeek'])['TotalPrice'].sum().unstack(fill_value=0)

In [29]:
display(revenue_ts_matrix.head())

Date,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30
StoreID,,,,,,,,,,,,,,,,,,,,,
1,6510.12,3683.01,1943.75,6051.44,1843.92,3888.35,1579.77,4618.63,4960.50,7140.28,...,4503.31,3775.15,3142.35,3069.50,2001.21,3300.05,1406.00,4323.99,5973.25,3427.51
2,12407.68,8714.52,3659.43,5417.80,4547.52,4946.04,5397.98,10396.10,9255.98,6484.20,...,4850.77,5644.12,8023.42,4158.62,7593.82,4972.22,3569.29,8593.40,8344.90,7264.55
3,4594.43,9152.73,3332.79,2319.01,8776.75,3279.88,5164.65,6550.87,6288.41,6604.59,...,954.04,2804.48,3812.66,5494.12,6223.77,8915.52,3108.03,4814.26,5849.46,8014.86
4,8111.78,11736.54,3085.22,2484.14,6289.78,4800.08,3176.76,6367.23,7887.70,1853.37,...,5214.55,4661.52,733.43,5793.67,5993.43,13406.79,5986.30,4068.70,3877.35,3441.99
5,7853.66,9878.44,4579.28,4345.18,5803.97,3078.29,8059.59,2530.69,8265.90,3895.92,...,7837.45,4446.95,2492.00,4238.35,4608.66,8516.14,6537.48,3476.44,4282.24,9586.15


In [13]:
# save to CSV
revenue_ts_matrix.to_csv('../data/sample-2/processed/revenue_ts_matrix.csv')
hourly_patterns.to_csv('../data/sample-2/processed/hourly_patterns.csv')
weekly_patterns.to_csv('../data/sample-2/processed/weekly_patterns.csv')

## Sequence Analysis Preparation

In [2]:
# Sort by customer and timestamp
df_sorted = df.sort_values(['CustomerID', 'Timestamp'])

# Create purchase sequences per customer
customer_sequences = df_sorted.groupby('CustomerID').agg({
    'ProductID': lambda x: list(x),
    'ProductCategory': lambda x: list(x),
    'StoreID': lambda x: list(x),
    'Timestamp': lambda x: list(x),
    'TotalPrice': lambda x: list(x)
}).reset_index()

# For store-focused sequence analysis
store_customer_sequences = df_sorted.groupby(['StoreID', 'CustomerID']).agg({
    'ProductID': lambda x: list(x),
    'ProductCategory': lambda x: list(x),
    'Timestamp': lambda x: list(x)
}).reset_index()

In [30]:
display(customer_sequences.head())

,CustomerID,ProductID,ProductCategory,StoreID,Timestamp,TotalPrice
0,Cust_0000,"[P125, P112, P140, P105, P143, P112, P123, P13...","[Electronics, Home Goods, Grocery, Electronics...","[4, 4, 3, 1, 6, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, ...","[2022-01-01 09:10:23, 2022-01-01 09:10:23, 202...","[373.44, 127.13, 17.22, 114.47, 284.02, 254.26..."
1,Cust_0001,"[P138, P138, P103, P114, P149, P109, P108, P10...","[Home Goods, Home Goods, Electronics, Apparel,...","[1, 4, 4, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, ...","[2022-01-03 15:31:41, 2022-01-04 17:28:22, 202...","[68.91, 68.91, 207.72, 168.5, 399.06, 40.91, 1..."
2,Cust_0002,"[P123, P143, P121, P103, P118, P149, P147, P12...","[Books, Books, Grocery, Electronics, Apparel, ...","[7, 1, 1, 1, 1, 1, 1, 3, 3, 7, 7, 7, 2, 7, 7, ...","[2022-01-08 16:22:50, 2022-01-29 14:42:42, 202...","[5.56, 142.01, 211.0, 207.72, 355.04, 399.06, ..."
3,Cust_0003,"[P127, P119, P148, P141, P145, P114, P117, P12...","[Books, Books, Books, Electronics, Grocery, Ap...","[8, 3, 3, 3, 1, 5, 5, 5, 5, 6, 2, 3, 3, 3, 3, ...","[2022-01-10 13:41:55, 2022-01-21 15:26:14, 202...","[212.94, 147.33, 169.0, 131.52, 270.34, 252.75..."
4,Cust_0004,"[P131, P106, P125, P132, P145, P141, P115, P14...","[Grocery, Books, Electronics, Electronics, Gro...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 7, 7, 7, 7, 6, ...","[2022-01-17 09:17:39, 2022-01-29 19:35:01, 202...","[169.99, 272.97, 231.77, 36.18, 290.58, 87.68,..."


In [3]:
display(store_customer_sequences.head())

,StoreID,CustomerID,ProductID,ProductCategory,Timestamp
0,1,Cust_0000,[P105],[Electronics],[2022-01-09 20:46:00]
1,1,Cust_0001,"[P138, P130, P127, P124, P140, P120, P104, P10...","[Home Goods, Electronics, Books, Electronics, ...","[2022-01-03 15:31:41, 2022-01-11 14:36:27, 202..."
2,1,Cust_0002,"[P143, P121, P103, P118, P149, P147, P111, P14...","[Books, Grocery, Electronics, Apparel, Apparel...","[2022-01-29 14:42:42, 2022-01-29 14:42:42, 202..."
3,1,Cust_0003,"[P145, P104]","[Grocery, Apparel]","[2022-01-26 17:11:45, 2022-03-25 10:03:10]"
4,1,Cust_0004,"[P118, P123]","[Apparel, Books]","[2022-04-07 10:20:51, 2022-04-07 10:20:51]"


In [17]:
customer_sequences.to_csv('../data/sample-2/processed/customer_sequences.csv', index=False)
store_customer_sequences.to_csv('../data/sample-2/processed/store_customer_sequences.csv', index=False)

## Market Basket Analysis Preparation

In [18]:
# Group by TransactionID to create baskets
transaction_baskets = df.groupby(['StoreID', 'TransactionID']).agg({
    'ProductID': lambda x: list(x),
    'ProductName': lambda x: list(x),
    'ProductCategory': lambda x: list(x)
}).reset_index()

# Store-specific baskets
store_baskets = {}
for store_id in df['StoreID'].unique():
    store_data = transaction_baskets[transaction_baskets['StoreID'] == store_id]
    store_baskets[store_id] = store_data['ProductID'].tolist()

# Convert to binary matrix format for each store (for algorithms like Apriori)
from sklearn.preprocessing import MultiLabelBinarizer

def create_basket_matrix(store_id):
    baskets = store_baskets[store_id]
    mlb = MultiLabelBinarizer()
    basket_matrix = mlb.fit_transform(baskets)
    return pd.DataFrame(basket_matrix, columns=mlb.classes_)

In [31]:
display(transaction_baskets.head())

,StoreID,TransactionID,ProductID,ProductName,ProductCategory
0,1,10000,[P105],[Product P105],[Electronics]
1,1,10001,"[P147, P132, P110]","[Product P147, Product P132, Product P110]","[Grocery, Electronics, Books]"
2,1,10002,"[P130, P107, P109]","[Product P130, Product P107, Product P109]","[Electronics, Electronics, Books]"
3,1,10003,"[P140, P144, P143, P144, P138, P112, P123, P122]","[Product P140, Product P144, Product P143, Pro...","[Grocery, Grocery, Books, Grocery, Home Goods,..."
4,1,10004,"[P131, P100, P104, P114, P117]","[Product P131, Product P100, Product P104, Pro...","[Grocery, Grocery, Apparel, Apparel, Electronics]"


In [27]:
transaction_baskets.to_csv('../data/sample-2/processed/transaction_baskets.csv', index=False)

## Store Specific Analysis

In [20]:
# Store-level summary statistics
store_summary = df.groupby('StoreID').agg({
    'TotalPrice': ['sum', 'mean', 'std'],
    'CustomerID': 'nunique',
    'TransactionID': 'nunique',
    'ProductID': 'nunique',
    'ProductCategory': lambda x: x.nunique(),
    'Quantity': 'sum',
    'DiscountAmount': 'sum'
}).reset_index()

# Category mix by store
category_mix = df.groupby(['StoreID', 'ProductCategory'])['TotalPrice'].sum().unstack(fill_value=0)
category_mix_pct = category_mix.div(category_mix.sum(axis=1), axis=0)

In [25]:
display(category_mix)

ProductCategory,Apparel,Books,Electronics,Grocery,Home Goods
StoreID,,,,,
1,155179.76,155622.44,245090.87,117282.13,78162.02
2,207235.09,220746.34,368836.23,154734.92,119459.68
3,211760.06,209267.07,357289.29,146134.45,109436.85
4,192791.31,174752.83,310117.36,130800.75,87549.29
5,191648.69,175603.62,283217.92,122010.77,95778.27
6,202637.56,192487.17,333961.33,141841.41,108621.30
7,181740.24,183716.40,310292.50,122162.86,91198.09
8,173590.21,175661.02,280161.32,116418.79,75439.71
